In [ ]:
import random
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import keras
import keras.backend as K

tf.logging.set_verbosity(tf.logging.ERROR)

In [ ]:
def rounded_binary_accuracy(y_true, y_pred):
    return K.mean(K.equal(K.round(y_true), K.round(y_pred)), axis=-1)

generator = keras.models.load_model("DCGAN-Cars_generator-50k.h5",
    custom_objects={'rounded_binary_accuracy': rounded_binary_accuracy})
generator.summary()

In [ ]:
r, c = 8,6
latent_dim = 128

noise = np.random.normal(0, 1, (r * c, latent_dim))
gen_imgs = generator.predict(noise)
gen_imgs = 0.5 * gen_imgs + 0.5

(X_train, Y_train), (X_test, Y_test) = keras.datasets.cifar10.load_data() #(label_mode='fine')
        
X_train = np.append(X_train, X_test, axis=0)
Y_train = np.append(Y_train, Y_test, axis=0)
        
X_train = X_train[Y_train.flatten() == 1]

idx = random.sample(range(len(X_train)), r*c)

plt.figure(figsize=(12,16))
cnt=0
for i in range(r):
    for j in range(c):
        plt.subplot(r,c,cnt+1)
        if j%2 == 0:
            plt.imshow(X_train[idx[cnt], :,:,:])
#            plt.imshow(gen_imgs[cnt, :,:,:])
        else:
            plt.imshow(gen_imgs[cnt, :,:,:])
        plt.axis('off')
        cnt += 1
        
plt.show()

In [ ]:
import json
with open('DCGAN-Cars.hist', 'r') as fp:
    history = json.load(fp)

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(1, 2, 1)

plt.plot(range(1, len(history['d_loss'])+1), history['d_loss'], linewidth=2, label='Discriminator loss')
plt.plot(range(1, len(history['g_loss'])+1), history['g_loss'], linewidth=2, label='Generator loss')
plt.legend()
plt.ylabel('D/G loss')
plt.xlabel('Steps')
        
plt.subplot(1, 2, 2)
plt.plot(range(1, len(history['d_acc'])+1), history['d_acc'], linewidth=2, label='Discriminator accuracy')
plt.legend()
plt.ylabel('Discriminator accuracy')
plt.xlabel('Steps')

plt.show()